In [5]:
# Code For Published Data Timeline

# Required Libraries
import os
import warnings
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
warnings.filterwarnings('ignore')

# !!!!Download Location!!!!
download_dir = r"C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim"

# Firefox Settings
options = Options()
options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
options.set_preference("browser.download.folderList", 2)
options.set_preference("browser.download.manager.showWhenStarting", False)
options.set_preference("browser.download.dir", download_dir)
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/vnd.ms-excel")
# Geckodriver Settings
service = Service(executable_path=r"C:\Users\oguzo\Desktop\Selenium\geckodriver.exe")
# WebDriver Opening
browser = webdriver.Firefox(service=service, options=options)
browser.get('https://www.tuik.gov.tr/Kurumsal/Veri_Takvimi')

# All years web scraper code
def scrape_page_data():
    time.sleep(2) 
    elements = browser.find_elements(By.CLASS_NAME, "timeline-item") # Calculating elements by class after 2 click
    page_data = []
    for element in elements:
        try:
            card_title = element.find_element(By.CLASS_NAME, "card-title").text
            card_body = element.find_element(By.CLASS_NAME, "box-item").text
            card_period = element.find_element(By.CLASS_NAME, "dt").text  
            link = element.find_element(By.TAG_NAME, "a").get_attribute("href") 
            page_data.append({
                'Kurum': card_title,
                'Açıklama': card_body,
                'Dönem ve Tarih': card_period,
                'Link': link  
            })
        except StaleElementReferenceException:
            print("StaleElementReferenceException hatası") 
    return page_data

# Data collection for all years
isimler = []
for year in reversed(range(2020, 2025)):
    # Clicking the year button panel
    first_click_for_year_selection = WebDriverWait(browser, 30).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div/div/div[1]/div[2]/div[2]/div[2]/button"))
    )
    browser.execute_script("arguments[0].click();", first_click_for_year_selection)
    year_button_xpath = f"/html/body/div/div/div[1]/div[2]/div[2]/div[2]/ul/li[{year-2019}]/a"
    try:
        year_button = WebDriverWait(browser, 30).until(
            EC.element_to_be_clickable((By.XPATH, year_button_xpath))
        )
        print(f"{year} yılı için veriler çekiliyor...")
        browser.execute_script("arguments[0].click();", year_button)

        WebDriverWait(browser, 30).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "timeline-item"))
        )
        
        time.sleep(3) 
        
        year_data = scrape_page_data()
        print(f"{year} yılı için {len(year_data)} eleman bulundu.")
        isimler += year_data  
        
    except TimeoutException:
        print("TimeoutException hatası")
browser.quit()

# Make a data frame for data
df = pd.DataFrame(isimler)

# Split 'Dönem ve Tarih' column to 2 pieces 'Dönem' and 'Tarih'. And drop the 'Dönem ve Tarih' column
df[['Dönem', 'Tarih']] = df['Dönem ve Tarih'].str.extract(r'Dönem:(.*)\nTarih:(.*)')
df.drop(columns=['Dönem ve Tarih'], inplace=True)

# Save as an excel file
excel_path = os.path.join(download_dir, 'Yayımlanan Veriler Takvimi.xlsx')
df.to_excel(excel_path, index=False)
print(f"Excel dosyası kaydedildi: {excel_path}")

2024 yılı için veriler çekiliyor...
2024 yılı için 3052 eleman bulundu.
2023 yılı için veriler çekiliyor...
2023 yılı için 3858 eleman bulundu.
2022 yılı için veriler çekiliyor...
2022 yılı için 3838 eleman bulundu.
2021 yılı için veriler çekiliyor...
2021 yılı için 3489 eleman bulundu.
2020 yılı için veriler çekiliyor...
2020 yılı için 3510 eleman bulundu.
Excel dosyası kaydedildi: C:\Users\oguzo\Desktop\Selenium\Tüik Takvim\TUIK-Takvim\a.xlsx
